# Create a Forecast plot given a stored model

### Notebook Configuration

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

### Imports

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["axes.axisbelow"] = True
import numpy as np
import pandas
import pandas as pd
from pandas import DataFrame, Index, Series, Timedelta, Timestamp
import matplotlib.dates as mdates

rng = np.random.default_rng()
np.set_printoptions()

import torch
import torchinfo
from torch import Tensor, jit, tensor
from torch.utils.data import DataLoader
from copy import deepcopy

### Enter Path and Name

In [ ]:
ID = (510, 16866)  # the Run_id / Experiment_id we want to plot.
RUN, EXP = ID
# files:  2021-11-12T00:52:10 2021-11-12T00:51:55 2021-11-12T00:51:48
# "checkpoints/2021-11-15T12:05:00/LinODEnet-0"
# "checkpoints/LinODEnet/KIWI_RUNS/skew_allways/2021-11-15T16:05:41/LinODEnet-0"
# "adam/2021-11-15T20:38:52/LinODEnet-0"
PATH = "checkpoints/LinODEnet/KIWI_RUNS/"
NAME = "SurrLoss+Sequential_Filter/2021-12-20T13:43:00/LinODEnet-50"
# the model checkpoint, should be a zip-archive created by torch.save / torch.jit.save
MODEL_FILE = PATH + NAME
DEVICE = torch.device("cpu")
DTYPE = torch.float32
NAN = tensor(float("nan"), dtype=DTYPE, device=DEVICE)
PRD_HORIZON = 900
OBS_HORIZON = 240
HORIZON = SEQLEN = OBS_HORIZON + PRD_HORIZON

## Intialize the task

In [ ]:
from tsdm.tasks import KIWI_RUNS_TASK

task = KIWI_RUNS_TASK(
    forecasting_horizon=PRD_HORIZON,
    observation_horizon=OBS_HORIZON,
)

## Load the Model

In [ ]:
model = torch.jit.load(MODEL_FILE, torch.device("cpu"))
torchinfo.summary(model, depth=5)

## Import Task

In [ ]:
assert ID in task.splits[(0, "test")][0].index
TRAINLOADER = task.dataloaders[(0, "train")]
EVALLOADER = task.dataloaders[(0, "test")]

dloader = EVALLOADER
dataset = dloader.dataset[ID]
preprocessor = deepcopy(dloader.preprocessor)
sampler = deepcopy(dloader.sampler[ID])
sampler.shuffle = False
LOADER = DataLoader(dataset, sampler=sampler)
slices = Series(LOADER)

ts = task.timeseries
ts = ts.loc[ID].astype("float32")

### Helper Function for Batch post-processing

In [ ]:
def prep_batch(batch: tuple[Tensor, Tensor]):
    """Get batch and create model inputs and targets."""
    T, X = batch
    targets = X[..., task.observation_horizon :, task.targets.index].clone()
    # assert targets.shape == (BATCH_SIZE, PRD_HORIZON, len(TASK.targets))
    originals = X.clone()
    inputs = X.clone()
    inputs[:, task.observation_horizon :, task.targets.index] = NAN
    inputs[:, task.observation_horizon :, task.observables.index] = NAN
    # assert inputs.shape == (BATCH_SIZE, HORIZON, NUM_DIM)
    return T, inputs, targets, originals

## Helper function to create the plot

In [ ]:
def make_plot(axes, task, batch):
    """Helper function to create plot automatically."""
    # batch = slices[0]
    times, inputs, targets, originals = (x.to(device="cpu") for x in prep_batch(batch))

    inputs[:, task.observation_horizon :, task.targets.index] = NAN
    inputs[:, task.observation_horizon :, task.observables.index] = NAN
    outputs = model(times, inputs)

    times = times[0].detach().cpu()
    inputs = inputs[0].detach().cpu()
    outputs = outputs[0].detach().cpu()
    targets = targets[0].detach().cpu()
    originals = originals[0].detach().cpu()

    times.shape, outputs.shape, inputs.shape, targets.shape, originals.shape
    reconstructed = preprocessor.decode((times, outputs)).astype("float32")

    for ax, (idx, target) in zip(axes.flatten(), task.targets.items()):
        color = next(ax._get_lines.prop_cycler)["color"]
        data = originals[:, idx]
        mask = ~np.isnan(data)
        ax.plot(
            reconstructed.index[: task.observation_horizon],
            reconstructed.iloc[: task.observation_horizon, idx],
            ls=":",
            lw=0.1,
            color="r",
        )
        ax.plot(
            reconstructed.index[task.observation_horizon - 1 :],
            reconstructed.iloc[task.observation_horizon - 1 :, idx],
            ls="-",
            lw=2,
            color="r",
        )
        print(reconstructed)

### Create the Raw data plot

In [ ]:
task.targets = task.observables[[3, 7, 9, 8]]

In [ ]:
%matplotlib widget

fig, axes = plt.subplots(
    nrows=2, ncols=2, sharex=True, figsize=(16, 8), constrained_layout=True
)

for ax, (idx, target) in zip(axes.flatten(), task.targets.items()):
    data = ts[target]
    times = ts.index.values
    mask = ~np.isnan(data)
    ax.plot(
        times[mask],
        data[mask],
        ls=":",
        lw=0.5,
        marker=".",
        ms=6,
    )
    ax.legend([f"{target} - observations"])

### Add the model Forecast Plots

In [ ]:
# grid = np.linspace(0, len(slices)-1, num=8, dtype=int)
grid = [300]
batches = slices.iloc[grid]

for batch in batches:
    make_plot(axes, task, batch)

fig.suptitle(f"Run{RUN}, Experiment {EXP}")

fig.savefig(f"pics/test-{NAME.replace(r'/', r'_')}_{PRD_HORIZON}.pdf")

In [ ]:
raise

## Creating per-channel plots for all channels

In [ ]:
def make_plot(ax, task, batch, idx):
    """Helper function to create plot automatically."""
    # batch = slices[0]
    times, inputs, targets, originals = (x.to(device="cpu") for x in prep_batch(batch))
    outputs = model(times, inputs)

    times = times[0].detach().cpu()
    inputs = inputs[0].detach().cpu()
    outputs = outputs[0].detach().cpu()
    targets = targets[0].detach().cpu()
    originals = originals[0].detach().cpu()

    times.shape, outputs.shape, inputs.shape, targets.shape, originals.shape
    reconstructed = preprocessor.decode((times, outputs)).astype("float32")

    T = reconstructed.index
    data = originals[:, idx]
    mask = ~np.isnan(data)
    ax.plot(
        T[: task.observation_horizon],
        reconstructed.iloc[: task.observation_horizon, idx],
        ls=":",
        lw=0.1,
        color="r",
    )
    ax.plot(
        T[task.observation_horizon - 1 :],
        reconstructed.iloc[task.observation_horizon - 1 :, idx],
        ls="-",
        lw=2,
        color="r",
    )

In [ ]:
columns = task.dataset.timeseries.columns

In [ ]:
from tsdm.utils.strings import snake2camel

In [ ]:
%matplotlib widget
ts = task.timeseries
# ts = ts[sorted(ts.columns, key=snake2camel)]
ts = ts.astype("float32").loc[ID]
T = ts.index  # ((ts.index - ts.index[0]) / Timedelta("1h")).values

fig, axes = plt.subplots(
    nrows=3, ncols=5, figsize=(16, 10), constrained_layout=True, sharex=True
)

for col, ax in zip(ts.columns, axes.flatten()):
    vals = ts[col]
    mask = pandas.notna(vals)
    ax.plot(
        T[mask],
        vals[mask],
        ls="-",
        lw=0.5,
        marker=".",
        ms=3,
    )
    ax.set_title(snake2camel(col))

In [ ]:
from tqdm.auto import tqdm

In [ ]:
grid = [300]
batches = slices.iloc[grid]

for batch in batches:
    for (idx, col), ax in tqdm(zip(enumerate(ts.columns), axes.flatten())):
        make_plot(ax, task, batch, idx)

In [ ]:
fig.savefig("pics/all_plots.png")